In [1]:
# Import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from IPython.display import clear_output
from matplotlib import cm, colors
from sklearn.datasets import fetch_olivetti_faces

# Noise Models

## FGSM

In [3]:
# import os
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt

# def fgsm_attack(image, epsilon, gradient):
#     # Calculate the sign of the gradient
#     signed_grad = tf.sign(gradient)
    
#     # Create the perturbed image by adding the scaled signed gradient to the original image
#     perturbed_image = image + epsilon * signed_grad
    
#     # Clip the pixel values of the perturbed image to the valid range [0, 1]
#     perturbed_image = tf.clip_by_value(perturbed_image, 0, 1)
    
#     return perturbed_image

# def apply_fgsm_attack_to_folder(folder_path, output_path, epsilon):
#     # Load or build your pre-trained model
#     model = tf.keras.applications.ResNet50(weights='imagenet')
    
#     # Create the output folder if it doesn't exist
#     if not os.path.exists(output_path):
#         os.makedirs(output_path)
    
#     # Iterate over the images in the folder
#     for filename in os.listdir(folder_path):
#         image_path = os.path.join(folder_path, filename)
        
#         # Load and preprocess the image
#         image = tf.io.read_file(image_path)
#         image = tf.image.decode_jpeg(image, channels=3)
#         image = tf.image.convert_image_dtype(image, tf.float32)
#         image = tf.image.resize(image, (224, 224))  # Resize the image to the desired dimensions
        
#         # Create a tensor variable from the image
#         image_tensor = tf.Variable(np.expand_dims(image, axis=0))
        
#         # Compute the gradient of the loss with respect to the input image
#         with tf.GradientTape() as tape:
#             tape.watch(image_tensor)
#             predictions = model(image_tensor)
#             top_prediction = tf.argmax(predictions, axis=1)
#             loss = tf.keras.losses.categorical_crossentropy(tf.one_hot(top_prediction, 1000), predictions)
        
#         gradients = tape.gradient(loss, image_tensor)
        
#         # Generate the adversarial example using FGSM
#         perturbed_image = fgsm_attack(image_tensor, epsilon, gradients)
        
#         # Get the perturbed image as a NumPy array
#         perturbed_image = np.squeeze(perturbed_image.numpy(), axis=0)
        
#         # Save the perturbed image to the output folder
#         output_filename = os.path.join(output_path, filename)
#         plt.imsave(output_filename, perturbed_image)
#         print(f"Saved perturbed image: {output_filename}")


## Gaussian

In [4]:
import cv2
import os

def add_gaussian_noise(image, mean, std_dev, epsilon):
    noise = np.random.normal(mean, std_dev, image.shape)
    noisy_image = np.clip(image + epsilon * noise, 0, 255).astype(np.uint8)
    return noisy_image

def gaussian_folder(input_path, output_path, epsilon):
    # Specify the mean, standard deviation, and epsilon value
    mean = 0
    std_dev = 30
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    # Iterate over the images in the folder
    for filename in os.listdir(input_path):
        image_path = os.path.join(input_path, filename)
        image = cv2.imread(image_path)
        # Add Gaussian noise to the image with the adjusted intensity
        noisy_image = add_gaussian_noise(image, mean, std_dev, epsilon)
        noisy_path = os.path.join(output_path, filename)
        cv2.imwrite(noisy_path, noisy_image)
    
    print("Done adding Gaussian mask to image.")


ModuleNotFoundError: No module named 'cv2'

In [ ]:
input_path = '/kaggle/input/human-faces-object-detection/images'
output_path = '/kaggle/working/Gaussian'
epsilon = .5

gaussian_folder(input_path, output_path, epsilon)


# Robust PCA denoise

In [ ]:
import cupy as cp

class Robust_PCA:

    def __init__(self, imax=1000, fthres=1e-6, verbose=False):
        self.imax = imax  # Maximum number of iterations
        self.fthres = fthres  # Convergence threshold
        self.verbose = verbose  # Print progress during iterations

    def shrink(self, X, tau):
        Y = cp.abs(X) - tau
        return cp.sign(X) * cp.maximum(Y, cp.zeros_like(Y))

    def SVT(self, X, tau):
        U, S, VT = cp.linalg.svd(X, full_matrices=0)
        out = U @ cp.diag(self.shrink(S, tau)) @ VT
        return out

    def fit(self, X):
        m, n = X.shape  # Store matrix dimensions
        mu = m * n / (4 * cp.sum(cp.abs(X.reshape(-1))))  # Calculate mu
        lambd = 1 / cp.sqrt(cp.max(cp.asarray([m, n])))  # Calculate lambda
        thres = self.fthres * cp.linalg.norm(X)  # Determine threshold (tolerance)

        # Initialize matrices L, S, and Y
        L = cp.zeros_like(X)
        S = cp.zeros_like(X)
        Y = cp.zeros_like(X)

        count = 0
        # Continue iterating while X ≈ L + S is above threshold or maximum number of iterations has been reached
        while (cp.linalg.norm(X - L - S) > thres) and (count < self.imax):
            if self.verbose:
                print(f'{cp.round(thres / cp.linalg.norm(X - L - S), 3)}% ...   ({count}/{self.imax})')

            # Update L, S, and Y
            L = self.SVT(X - S + 1 / mu * Y, 1 / mu)
            S = self.shrink(X - L + 1 / mu * Y, lambd / mu)
            Y = Y + mu * (X - L - S)
            count += 1

        # Print number of iterations required
        if self.verbose:
            if count == self.imax:
                print(f'No convergence after {count} iterations')
            else:
                print(f'Solution found after {count} iterations')

        self.L_ = L
        self.S_ = S

        return self


In [ ]:
import cv2
list_img = os.listdir('/kaggle/working/Gaussian')
# list_img
num_images = len(list_img)
image_size = 128  # Set the desired size for the flattened images

r_matrix = np.zeros((num_images, image_size * image_size), dtype=np.uint8)
g_matrix = np.zeros((num_images, image_size * image_size), dtype=np.uint8)
b_matrix = np.zeros((num_images, image_size * image_size), dtype=np.uint8)

img_org_size = []
for i, img in enumerate(list_img):
    img_path = os.path.join('/kaggle/working/Gaussian', img)
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    img_org_size.append((height, width))
    
    b, g, r = cv2.split(img)   
    b = cv2.resize(b, (image_size, image_size))
    flattened_b = b.flatten()
    b_matrix[i] = flattened_b
    g = cv2.resize(g, (image_size, image_size))
    flattened_g = g.flatten()
    g_matrix[i] = flattened_g
    r = cv2.resize(r, (image_size, image_size))
    flattened_r = r.flatten()
    r_matrix[i] = flattened_r
#     print(img.shape)

In [ ]:
r_matrix = r_matrix / np.max(r_matrix)
g_matrix = g_matrix / np.max(g_matrix)
b_matrix = b_matrix / np.max(b_matrix)

In [ ]:
# Perform robust PCA on the digits-matrix
# Example usage
noise_b_gpu, noise_g_gpu, noise_r_gpu = cp.asarray(b_matrix), cp.asarray(g_matrix), cp.asarray(r_matrix)

rpcadm = Robust_PCA(imax=300, fthres=1e-7, verbose=True)

print('Computing Sparse PCA for blue matrix...')
rpcadm.fit(noise_b_gpu)
L_b, S_b = cp.asnumpy(rpcadm.L_), cp.asnumpy(rpcadm.S_)
print('Done. Computing Sparse PCA for green matrix...')

rpcadm.fit(noise_g_gpu)
L_g, S_g = cp.asnumpy(rpcadm.L_), cp.asnumpy(rpcadm.S_)
print('Done. Computing Sparse PCA for red matrix...')

rpcadm.fit(noise_r_gpu)# Provide your input data matrix X
L_r, S_r = cp.asnumpy(rpcadm.L_), cp.asnumpy(rpcadm.S_)# Obtained low-rank component
print('Done')
  # Obtained sparse component

In [ ]:
fig, axs = plt.subplots(5, 3, figsize = (7, 12))
# image_size = 32

for k in range(5):
    red_image = r_matrix[k].reshape(image_size,image_size)
    green_image = g_matrix[k].reshape(image_size ,image_size)
    blue_image = b_matrix[k].reshape(image_size, image_size)
    # Stack the individual color channels to create an RGB image
    rgb_image = np.stack((red_image, green_image, blue_image), axis=2)
    rgb_image = cv2.resize(rgb_image, (img_org_size[k][0], img_org_size[k][1]))
    rgb_image_scaled = (rgb_image * 255).astype(np.uint8)
    axs[k, 0].imshow(rgb_image_scaled)
    
    noise_red = S_r[k].reshape(image_size,image_size)
    noise_green = S_g[k].reshape(image_size ,image_size)
    noise_blue = S_b[k].reshape(image_size, image_size)
    # Stack the individual color channels to create an RGB image
    rgb_noise_image = np.stack((noise_red, noise_green, noise_blue), axis=2)
    rgb_noise_image = cv2.resize(rgb_noise_image, (img_org_size[k][0], img_org_size[k][1]))
    rgb_noise_image_scaled = (rgb_noise_image * 255).astype(np.uint8)
    axs[k, 1].imshow(rgb_noise_image_scaled)
    
    Lr_image = L_r[k].reshape(image_size,image_size)
    Lg_image = L_g[k].reshape(image_size ,image_size)
    Lb_image = L_b[k].reshape(image_size, image_size)
    # Stack the individual color channels to create an RGB image
    L_image = np.stack((Lr_image, Lg_image, Lb_image), axis=2)
    L_image = cv2.resize(L_image, (img_org_size[k][0], img_org_size[k][1]))
    L_image_scaled = (L_image * 255).astype(np.uint8)
    axs[k, 2].imshow(L_image_scaled)

    for i in range(3):
        axs[k,i].set_xticks([])
        axs[k,i].set_yticks([])
        
    if k == 0:
        axs[k,0].set_title('Noise image', size = 6)
        axs[k,1].set_title('Noise matrix', size = 6)
        axs[k,2].set_title('Low resolution image', size = 6)
#     axs[i,1].set_yticks([])
plt.tight_layout()
plt.savefig('Test.jpeg', dpi = 400)
plt.show()